In [2]:
from SMT_model import *
from time import time as time_clock
from z3 import IntNumRef
import json

In [3]:
#Demo with instance 01
Path = ('../Instances/inst01.dat')

timeout = 300


In [4]:
#Read the file from the path
with open(Path, 'r') as file:
    data = file.read()

instance = data_parsing(data)
print(instance)

{'m': 2, 'n': 6, 'l': [15, 10], 's': [3, 2, 6, 5, 4, 4], 'D': [[0, 3, 4, 5, 6, 6, 2], [3, 0, 1, 4, 5, 7, 3], [4, 1, 0, 5, 6, 6, 4], [4, 4, 5, 0, 3, 3, 2], [6, 7, 8, 3, 0, 2, 4], [6, 7, 8, 3, 2, 0, 4], [2, 3, 4, 3, 4, 4, 0]]}


In [5]:
def run_smt(instance, timeout, sb=False):
    generation_start_time = time_clock()

    # Build the SMT model
    o, x, max_distance = stm_model(instance, sb)
    generation_duration = time_clock() - generation_start_time
    o.set("timeout", int(timeout - generation_duration) * 1000)

    # Minimize the objective
    obj = o.minimize(max_distance)
    res = o.check()  # Check satisfiability
    final_time = int(time_clock() - generation_start_time)

    if res == sat:
        try:
            # Format the solution if satisfiable
            result_formatted = format_solution(instance, o.model(), x)
            return result_formatted, True, obj.value(), final_time
        except Exception as e:
            return str(e), False, obj.value()
    elif res == unknown:
        try:
            model = o.model()
            if model:  # Check if a model exists
                # Format the partial solution
                result_formatted = format_solution(instance, model, x)
                best_objective = model.eval(max_distance, model_completion=True)
                return (
                    result_formatted,
                    False,
                    best_objective.as_long() if best_objective.is_int() else float(best_objective.as_decimal(5)),
                    final_time
                )
            else:
                # No model available, return fallback message
                return "No solution found", False, None
        except Exception as e:
            return f"unknown\nError retrieving model: {e}", False, None
    elif res == unsat:
        return "unsat", False, None
    else:
        return "unknown", False, None

In [10]:
print(run_smt(instance, timeout, False))

sat
[x_7_6 = 13,
 x_2_3 = 13,
 x_5_5 = 13,
 x_0_4 = 13,
 x_2_4 = 13,
 x_6_11 = 13,
 x_8_1 = 13,
 x_9_2 = 13,
 x_3_6 = 13,
 x_3_2 = 13,
 x_9_1 = 0,
 x_5_2 = 13,
 distances = Store(Store(Store(Store(Store(Store(Store(Store(Store(Store(Store(Store(Store(K(Int,
                                        Store(Store(Store(Store(Store(Store(Store(...,
                                        ...,
                                        ...),
                                        12,
                                        65),
                                        2,
                                        137),
                                        13,
                                        108),
                                        9,
                                        79),
                                        3,
                                        49),
                                        10,
                                        59)),
                              

In [11]:
def all_solutions(solvers, solutions):
    def convert(obj):
        if isinstance(obj, IntNumRef):
            return obj.as_long()
        if isinstance(obj, list):
            return [convert(i) for i in obj]
        return obj
    
    output_data = {}
    sol_index=0
    for solver in solvers:
        solution = solutions[sol_index]
        if solution == "unsat":
            print("Problem is unsatisfiable for solver", solver)
        elif solution == "unknown":
            print("Problem is unknown for solver", solver)
        elif solution[0] == "No solution found":
            print("No solution found for solver", solver)
        else:
            output_data.update({
                solver:
                    {
                    "time": convert(solution[3]),
                    "optimal": convert(solution[1]),
                    "obj": convert(solution[2]),
                    "sol": convert(solution[0])
                    }
                })
        sol_index += 1
        
    return output_data

In [12]:
def save_solution(solvers ,solutions, path):
    output_data = all_solutions(solvers, solutions)
    with open(path, 'w') as file:
        if output_data != []:
            print(output_data)
            json.dump(output_data, file, indent=4)

In [13]:
#Create a function that runs the SMT model for all the instances and saves the solutions in json files.
def run_all_instances():
    for i in range(1, 2):
        if i < 10:
            Path = f'../Instances/inst0{i}.dat'
        else:
            Path = f'../Instances/inst{i}.dat'
        if i < 30:
            with open(Path, 'r') as file:
                data = file.read()
            instance = data_parsing(data)
            solution_smt = run_smt(instance, timeout, False)
            solution_smt_sb = run_smt(instance, timeout, True)
            save_solution(["smt", "smt_sb"], [solution_smt, solution_smt_sb], f'res/SMT/inst{i}/inst{i}.json')

In [14]:
run_all_instances()

{'smt': {'time': 1, 'optimal': True, 'obj': 14, 'sol': [[4, 3, 1], [2, 5, 6]]}, 'smt_sb': {'time': 0, 'optimal': True, 'obj': 14, 'sol': [[4, 3, 1], [2, 5, 6]]}}
